# 🧬 HBV Tahoe-x1 임베딩 분석

**목적**: HBV 데이터에 Tahoe-x1 파이프라인 적용하여 세포 임베딩 추출

---

## 📋 워크플로우

| STEP | 내용 |
|------|------|
| 1 | 환경 설정 |
| 2 | HBV 데이터 로드 |
| 3 | Tahoe-x1 모델 로드 |
| 4 | 데이터 전처리 |
| 5 | 임베딩 추출 |
| 6 | 클러스터링 |
| 7 | 세포 유형 식별 |
| 8 | 결과 저장 |

⚠️ **사전 조건**: `HBV_data_download.ipynb`를 먼저 실행하여 데이터 준비

In [ ]:
# @title STEP 1: 환경 설정
import os, sys, platform

if platform.system() == 'Windows':
    raise RuntimeError('🛑 Colab에서 실행하세요!')

from google.colab import drive
drive.mount('/content/drive')

PROJECT_PATH = '/content/drive/MyDrive/tahoe-x1'
DATA_PATH = f'{PROJECT_PATH}/viral_hepatitis/raw_data'
OUTPUT_PATH = f'{PROJECT_PATH}/viral_hepatitis/analysis_results'
os.makedirs(OUTPUT_PATH, exist_ok=True)
os.chdir(PROJECT_PATH)
sys.path.insert(0, PROJECT_PATH)

print('📦 패키지 설치 중...')
!pip install scanpy anndata einops omegaconf igraph leidenalg boto3 mosaicml transformers -q
!pip install -e . -q

print(f'\n✅ STEP 1 완료!')
print(f'📁 데이터: {DATA_PATH}')
print(f'📁 결과: {OUTPUT_PATH}')

In [ ]:
# @title STEP 2: HBV 데이터 로드
import scanpy as sc
import os

# 사용 가능한 데이터 파일 확인
print('📁 사용 가능한 데이터 파일:')
h5ad_files = [f for f in os.listdir(DATA_PATH) if f.endswith('.h5ad')]
for f in h5ad_files:
    print(f'   - {f}')

# 데이터 로드 (시뮬레이션 데이터 또는 실제 데이터)
DATA_FILE = 'simulated_hbv_pbmc.h5ad'  # @param {type:"string"}

print(f'\n📊 데이터 로딩: {DATA_FILE}')
adata = sc.read_h5ad(f'{DATA_PATH}/{DATA_FILE}')

print(f'\n✅ STEP 2 완료!')
print(f'   세포 수: {adata.n_obs}')
print(f'   유전자 수: {adata.n_vars}')
if 'hbv_status' in adata.obs.columns:
    print(f'\n📊 HBV 상태별 세포 수:')
    print(adata.obs['hbv_status'].value_counts())

In [ ]:
# @title STEP 3: Tahoe-x1 모델 로드
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from datetime import datetime

from tahoe_x1.model import ComposerTX
from tahoe_x1.utils.util import loader_from_adata

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
MODEL_SIZE = '70m'

print(f'🖥️ Device: {DEVICE}')
print(f'🚀 Tahoe-x1 {MODEL_SIZE} 모델 로딩 중...')

model, vocab, model_cfg, collator_cfg = ComposerTX.from_hf(
    repo_id='tahoebio/Tahoe-x1',
    model_size=MODEL_SIZE,
    attn_impl='torch'
)
model.to(DEVICE).eval()
torch.cuda.empty_cache()

print(f'\n✅ STEP 3 완료!')

In [ ]:
# @title STEP 4: 데이터 전처리
print('🧬 데이터 전처리 시작...')

# 원본 보존
adata_raw = adata.copy()

# 기본 필터링
sc.pp.filter_genes(adata, min_cells=3)

# Ensembl ID 변환 (gene_ids 컬럼이 있는 경우)
if 'gene_ids' in adata.var.columns:
    adata.var['gene_symbol'] = adata.var_names.copy()
    adata.var_names = [g.split('.')[0] for g in adata.var['gene_ids'].astype(str)]
    adata.var_names_make_unique()

# id_in_vocab 추가
adata.var['id_in_vocab'] = [vocab.token_to_index.get(g, -1) for g in adata.var_names]
valid_mask = adata.var['id_in_vocab'] >= 0
adata = adata[:, valid_mask].copy()

print(f'\n✅ STEP 4 완료!')
print(f'   전처리 후: {adata.shape}')

In [ ]:
# @title STEP 5: 임베딩 추출
BATCH_SIZE = 4

print(f'🧠 임베딩 추출 (배치: {BATCH_SIZE})')

loader = loader_from_adata(
    adata=adata,
    collator_cfg=collator_cfg,
    vocab=vocab,
    batch_size=BATCH_SIZE,
    num_workers=2,
    prefetch_factor=2
)

all_embeddings = []
with torch.no_grad():
    for batch in tqdm(loader, desc='추출 중'):
        batch = {k: v.to(DEVICE) if isinstance(v, torch.Tensor) else v 
                 for k, v in batch.items()}
        output = model(batch, skip_decoders=True)
        all_embeddings.append(output['cell_emb'].cpu().numpy())
        del output, batch
        torch.cuda.empty_cache()

adata.obsm[f'X_tahoe_{MODEL_SIZE}'] = np.vstack(all_embeddings)
adata_raw.obsm[f'X_tahoe_{MODEL_SIZE}'] = adata.obsm[f'X_tahoe_{MODEL_SIZE}']

print(f'\n✅ STEP 5 완료!')
print(f'   임베딩 차원: {adata.obsm[f"X_tahoe_{MODEL_SIZE}"].shape}')

In [ ]:
# @title STEP 6: UMAP 및 클러스터링
print('🗺️ UMAP 및 클러스터링...')

sc.pp.neighbors(adata_raw, use_rep=f'X_tahoe_{MODEL_SIZE}')
sc.tl.umap(adata_raw)
sc.tl.leiden(adata_raw, resolution=0.5)

# HBV 상태별 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

sc.pl.umap(adata_raw, color='leiden', ax=axes[0], show=False,
           title='Cell Clusters', legend_loc='on data')

if 'hbv_status' in adata_raw.obs.columns:
    sc.pl.umap(adata_raw, color='hbv_status', ax=axes[1], show=False,
               title='HBV Status')
else:
    axes[1].text(0.5, 0.5, 'No HBV status data', ha='center', va='center')
    axes[1].set_title('HBV Status')

plt.tight_layout()
plt.savefig(f'{OUTPUT_PATH}/hbv_umap_overview.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\n✅ STEP 6 완료!')

In [ ]:
# @title STEP 7: 세포 유형 식별
print('🔬 세포 유형 분석 중...\n')

# PBMC 마커 유전자
markers = {
    'CD4+ T cells': ['CD3D', 'CD4', 'IL7R'],
    'CD8+ T cells': ['CD3D', 'CD8A', 'CD8B'],
    'B cells': ['MS4A1', 'CD79A', 'CD79B'],
    'NK cells': ['GNLY', 'NKG7'],
    'CD14+ Monocytes': ['CD14', 'LYZ', 'S100A8'],
    'FCGR3A+ Monocytes': ['FCGR3A', 'MS4A7'],
    'Dendritic cells': ['FCER1A', 'CST3'],
    'Platelets': ['PPBP', 'PF4']
}

# 탈진 마커 (HBV 관련)
exhaustion_markers = ['PDCD1', 'HAVCR2', 'LAG3', 'CTLA4', 'TIGIT']

# 클러스터별 세포 유형 식별
cluster_types = {}
for cluster in sorted(adata_raw.obs['leiden'].unique(), key=int):
    cluster_cells = adata_raw[adata_raw.obs['leiden'] == cluster]
    best_type, best_score = 'Unknown', 0
    
    for cell_type, genes in markers.items():
        valid_genes = [g for g in genes if g in adata_raw.var_names]
        if valid_genes:
            score = cluster_cells[:, valid_genes].X.toarray().mean() if hasattr(cluster_cells[:, valid_genes].X, 'toarray') else cluster_cells[:, valid_genes].X.mean()
            if score > best_score:
                best_score, best_type = score, cell_type
    
    cluster_types[cluster] = best_type
    print(f'  클러스터 {cluster:>2}: {best_type:<20} ({len(cluster_cells)}개 세포)')

adata_raw.obs['cell_type'] = adata_raw.obs['leiden'].map(cluster_types)

# 시각화
fig, ax = plt.subplots(figsize=(10, 8))
sc.pl.umap(adata_raw, color='cell_type', ax=ax, show=False, title='Cell Types')
plt.savefig(f'{OUTPUT_PATH}/hbv_cell_types.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\n✅ STEP 7 완료!')

In [ ]:
# @title STEP 8: 결과 저장
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_file = f'{OUTPUT_PATH}/hbv_tahoe_analysis_{timestamp}.h5ad'

adata_raw.write_h5ad(output_file)

print('=' * 50)
print('🎉 Tahoe-x1 분석 완료!')
print('=' * 50)
print(f'\n📁 저장된 파일:')
print(f'   📊 {output_file}')
print(f'   🖼️ {OUTPUT_PATH}/hbv_umap_overview.png')
print(f'   🖼️ {OUTPUT_PATH}/hbv_cell_types.png')
print(f'\n📋 분석 요약:')
print(f'   세포 수: {adata_raw.n_obs}')
print(f'   클러스터 수: {adata_raw.obs["leiden"].nunique()}')
print('\n✅ 다음 단계: Step2_HBV_group_comparison.ipynb 실행')